# Visualizing the 3d structures

To visualize the true and reconstructed 3d structures:
1. Pad the true structure so that the cell has the same shape as the reconstructed structure.
2. Save the structures as matlab files.
3. Use 'plot3d.m' within matlab for the visualization.

**Note**:
Some parts of this script assume the existence of the file containing the reconstructed crystal cell.

In [ ]:
import numpy as np
from scipy import ndimage, io, stats

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cell_orig = np.load('../cell_faceted_2.npy') * 0.5
cell_orig.shape

In [ ]:
cell_padded = np.pad(cell_orig, [[18, 18],[50, 50],[25, 25]], mode='constant')
cell_padded.shape

In [ ]:
nonzero_indices = np.transpose(np.where(np.abs(cell_padded) > 0))
raveled_indices = np.ravel_multi_index(nonzero_indices.T, cell_padded.shape)
nonzero_vals = cell_padded.flatten()[raveled_indices]

nonzero_vals = nonzero_vals * np.exp(-1j * stats.circmean(np.angle(nonzero_vals)))

cell_padded_temp = np.zeros(cell_padded.size, dtype='complex64')
cell_padded_temp[raveled_indices] = nonzero_vals
cell_padded = cell_padded_temp.reshape(cell_padded.shape)

In [ ]:
var_true = np.pad(cell_orig, [[1,1],[10,10],[10,10]], mode='constant')
var_true.shape

In [ ]:
var_reconstructed =  np.load('../reconstructed_faceted_cell_rotated_ki_and_kf_batch_400_stepsize_0.01.npy')
var_padded = np.pad(var_reconstructed, [[18,18], [54,54], [22,22]], mode='constant')
print(var_padded.shape)

recons_nonzero_vals = (np.ravel(var_padded)[raveled_indices])

# Removing the global phase shift
recons_nonzero_vals = recons_nonzero_vals * np.exp(-1j * scipy.stats.circmean(np.angle(recons_nonzero_vals)))
var_reconstructed = var_reconstructed * np.exp(-1j * scipy.stats.circmean(np.angle(recons_nonzero_vals)))
var_padded = var_padded * np.exp(-1j * scipy.stats.circmean(np.angle(recons_nonzero_vals)))

In [ ]:
var_reconstructed[np.abs(var_reconstructed)==0] = 0 + 0j
var_reconstructed = np.roll(var_reconstructed, 1, axis=0)
var_true[np.abs(var_true==0)] = 0 + 0j

In [ ]:
io.savemat('cell_true_var', dict(cell_true=var_true))
io.savemat('cell_recons_var', dict(cell_recons=var_reconstructed))